In [2]:
import numpy as np
import pandas as pd  # Added import for pandas

# Create a sample DataFrame first
# Replace this with your actual data loading code
df = pd.DataFrame({
    'TransactionID': range(1000, 1020),
    'Amount': np.random.randint(100, 100000, 20),
    'Timestamp': pd.date_range(start='2023-01-01', periods=20, freq='H'),
    'DeviceID': np.random.choice(['dev1', 'dev2', 'dev3', 'dev4'], 20),
    'UserID': np.random.choice(['user1', 'user2', 'user3'], 20)
})

# Transaction features
df['LogAmount'] = np.log1p(df['Amount'])
df['Hour'] = df['Timestamp'].dt.hour
df['IsNight'] = df['Hour'].apply(lambda x: 1 if x in [22,23,0,1,2,3,4,5] else 0)

# Behavioral features
df['DeviceTxnCount'] = df.groupby('DeviceID')['TransactionID'].transform('count')

# Rule-based flags
df['RuleHighValue'] = df['Amount'].apply(lambda x: 1 if x > 50000 else 0)
df['TxnWindow'] = df['Timestamp'].dt.floor('2T')
df['TxnCountWindow'] = df.groupby(['UserID','TxnWindow'])['TransactionID'].transform('count')
df['RuleRapidFire'] = df['TxnCountWindow'].apply(lambda x: 1 if x > 5 else 0)

C:\Users\naman\AppData\Local\Temp\ipykernel_13568\1260613991.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'Timestamp': pd.date_range(start='2023-01-01', periods=20, freq='H'),
C:\Users\naman\AppData\Local\Temp\ipykernel_13568\1260613991.py:24: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df['TxnWindow'] = df['Timestamp'].dt.floor('2T')


In [6]:
import numpy as np

# Log-transform amount
df['LogAmount'] = np.log1p(df['Amount'])

# Time features
df['Hour'] = df['Timestamp'].dt.hour
df['IsNight'] = df['Hour'].apply(lambda x: 1 if x in [22,23,0,1,2,3,4,5] else 0)

# Daily transaction count per user
df['DailyTxnCount'] = df.groupby([df['UserID'], df['Timestamp'].dt.date])['TransactionID'].transform('count')

In [7]:
# Device usage frequency
df['DeviceTxnCount'] = df.groupby('DeviceID')['TransactionID'].transform('count')

# Rapid-fire transactions (within 2 minutes window)
df['TxnWindow'] = df['Timestamp'].dt.floor('2T')
df['TxnCountWindow'] = df.groupby(['UserID','TxnWindow'])['TransactionID'].transform('count')
df['RuleRapidFire'] = df['TxnCountWindow'].apply(lambda x: 1 if x > 5 else 0)

# High-value flag
df['RuleHighValue'] = df['Amount'].apply(lambda x: 1 if x > 50000 else 0)

C:\Users\naman\AppData\Local\Temp\ipykernel_13568\2861447178.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df['TxnWindow'] = df['Timestamp'].dt.floor('2T')


In [12]:
# Now that we know the available columns, let's modify the code

# Import necessary libraries if not already imported
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Print the columns to verify
print(df.columns)

# Since 'TransactionType' doesn't exist, we'll work with the columns that do exist
# Let's encode the categorical columns that are available: 'DeviceID'

# Encode DeviceID
le_dev = LabelEncoder()
df['DeviceID_encoded'] = le_dev.fit_transform(df['DeviceID'])

# If you need to encode UserID as well (assuming it's categorical)
le_user = LabelEncoder()
df['UserID_encoded'] = le_user.fit_transform(df['UserID'])

# For binary features like 'IsNight' and 'RuleHighValue', 'RuleRapidFire'
# they might already be encoded as 0/1, but if not:
binary_columns = ['IsNight', 'RuleHighValue', 'RuleRapidFire']
for col in binary_columns:
    if df[col].dtype == 'object':  # If they're not already numeric
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col])

# If you need one-hot encoding for any columns with multiple categories
# For example, if Hour has multiple discrete values:
# df = pd.get_dummies(df, columns=['Hour'], drop_first=True)

# Display the DataFrame with the new encoded columns
print(df.head())

Index(['TransactionID', 'Amount', 'Timestamp', 'DeviceID', 'UserID',
       'LogAmount', 'Hour', 'IsNight', 'DeviceTxnCount', 'RuleHighValue',
       'TxnWindow', 'TxnCountWindow', 'RuleRapidFire', 'DailyTxnCount'],
      dtype='object')
   TransactionID  Amount           Timestamp DeviceID UserID  LogAmount  Hour  \
0           1000   77240 2023-01-01 00:00:00     dev1  user2  11.254686     0   
1           1001   67195 2023-01-01 01:00:00     dev3  user2  11.115369     1   
2           1002   55774 2023-01-01 02:00:00     dev1  user3  10.929081     2   
3           1003   55953 2023-01-01 03:00:00     dev2  user2  10.932285     3   
4           1004   33663 2023-01-01 04:00:00     dev3  user2  10.424184     4   

   IsNight  DeviceTxnCount  RuleHighValue           TxnWindow  TxnCountWindow  \
0        1               6              1 2023-01-01 00:00:00               1   
1        1               5              1 2023-01-01 01:00:00               1   
2        1               6      

In [15]:
from sklearn.preprocessing import StandardScaler

feature_cols = [
    'LogAmount','Hour','IsNight','DailyTxnCount','DeviceTxnCount',
    'RuleHighValue','RuleRapidFire','DeviceID_encoded'
] + [col for col in df.columns if col.startswith('MerchantCategory_')]

X = df[feature_cols]

# Scale numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)